In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import sqlite3

# Connect to Database
con = sqlite3.connect('../data/raw/database.db') 
cur = con.cursor()

In [3]:
# read column names
columns = []
for idx, name, col_type, *args in cur.execute("PRAGMA table_info('listings');").fetchall():
    columns.append(name)
columns[:5]

['user_id', 'id', 'uri', 'title', 'price']

In [4]:
# read table content
sql_query = pd.read_sql('SELECT * FROM listings', con)

# Convert SQL to DataFrame
df = pd.DataFrame(sql_query, columns=columns)

df.head()

,user_id,id,uri,title,price,content,imgs,refresh,category,beds,...,district,district_id,width,length,advertiser_type,create_time,has_extended_details,daily_rentable,createdAt,updatedAt
0,10065,4331,شارع-الياقوت-الدار-البيضاء-الرياض,"أرض للبيع في شارع الياقوت, الدار البيضاء, الرياض",1958400.0,تجاريه ش ياقوت ٦٠ مساحه ٨١٦ تقريبا,"[""000000000_1627687178830.jpg""]",1672495809,2,0.0,...,حي الدار البيضاء,424,25.0,32.0,None,1415294130,NaN,0.0,2023-01-01 14:13:28.553 +00:00,2023-01-01 14:13:28.553 +00:00
1,11005,4837,طريق-الملك-عبدالله-بن-عبدالعزيز-الفرعي-المغرزا...,أرض للبيع في طريق الملك عبدالله بن عبدالعزيز ا...,15078000.0,الارض على طريق الملك عبدالله على شارعين مظاهره...,"[""000000000_1627687181813.jpg""]",1672040788,2,0.0,...,حي المغرزات,566,NaN,NaN,None,1416054461,NaN,0.0,2023-01-01 14:52:38.169 +00:00,2023-01-01 14:52:38.169 +00:00
2,8017,5498,5-المملكة-العربية-السعودية-5498,أرض للبيع في 5 المملكة العربية السعودية,8057700.0,الارض استثماريه في محافظة البرك طريق جده جازان...,"[""506610555_1417214243205.jpg"",""506610555_1417...",1671819056,2,0.0,...,حي الروضة,4100,NaN,NaN,None,1417220044,NaN,0.0,2023-01-01 15:01:52.351 +00:00,2023-01-01 15:01:52.351 +00:00
3,13810,6520,شارع-Saher-Al-Garbee-St-ذهبان,"أرض للإيجار في شارع الهزاعية, حي العشيرية, جدة",50000.0,السلام عليكم ورحمة الله وبركاته\nوكل عام وانتم...,"[""504121665_1418232181483.jpg"",""504121665_1418...",1672190424,15,0.0,...,حي الرياض,3490,20.0,20.0,None,1418232587,NaN,0.0,2023-01-01 14:43:23.326 +00:00,2023-01-01 14:43:23.326 +00:00
4,5234,8847,طريق-الملك-خالد-حي-الثقبة-الخبر-الخبر-8847,أرض للبيع في طريق الملك خالد ، حي الثقبة ، الخ...,3000000.0,ارض للبيع لبناء شقق فندقيه او عمارات سكنيه خلف...,"[""567772202_1632148243568.png"",""000052346_1661...",1672253180,2,0.0,...,حي الثقبة,1060,NaN,NaN,None,1420232812,NaN,0.0,2023-01-01 14:38:46.004 +00:00,2023-01-01 14:38:46.004 +00:00


### Data cleaning 

In [5]:
# look at None statistics
nan_info = df.isna().sum()
nan_info

user_id                      0
id                           0
uri                          0
title                       21
price                        0
content                    230
imgs                         0
refresh                      0
category                     0
beds                    252609
livings                 252119
wc                      254128
area                       577
type                    502773
street_width             18982
age                     253127
last_update                  0
street_direction         94891
ketchen                 277809
ac                      555898
furnished               238949
location.lat                 0
location.lng                 0
path                         0
user.review             118066
user.img                222565
user.name                 6285
user.phone                   0
user.iam_verified         3714
user.rega_id            338325
native.logo             663944
native.title            663944
native.i

In [6]:
# colums to remove entirely
remove_cols = [
    # too few values
    'native.logo', 'native.title', 'native.image', 'native.description', 'native.external_url',
    
    # too unique/can lead to bias
    'user.phone', 'user.rega_id', 'user_id', 'id',
    
    # invalid data type
    'user.img', 'uri', 'user.name', 'city', 'district', 'imgs', 'path',
    
    # invalid data type, but might be considered usefull with interpretable NLP
    'content', 'title',
    
    # website time data
    'last_update', 'create_time', 'createdAt', 'updatedAt', 'refresh',
    
    # strongly correlated with any other column
    'width', 'length',
    
    # unexplainable features
    'type'
]

In [7]:
df_dropped = df.drop(columns=remove_cols)
df_dropped.isna().sum()

price                        0
category                     0
beds                    252609
livings                 252119
wc                      254128
area                       577
street_width             18982
age                     253127
street_direction         94891
ketchen                 277809
ac                      555898
furnished               238949
location.lat                 0
location.lng                 0
user.review             118066
user.iam_verified         3714
rent_period             571232
city_id                      0
district_id                  0
advertiser_type          23932
has_extended_details    653524
daily_rentable          315493
dtype: int64

In [8]:
def print_stats(column_name: str):
    print('----', column_name, '----')
    print(df[column_name].mean(), df[column_name].median())
    print(df[column_name].unique())
    

for column_name in df_dropped.columns:
    if column_name == 'advertiser_type':
        continue
    print_stats(column_name)

---- price ----
3986665.0763846464 1000000.0
[ 1958400. 15078000.  8057700. ...  3939488.  1871500.  4367190.]
---- category ----
4.148884095995759 3.0
[ 2 15 16  3  7  8  6 12  1  9 19 13 10 21 14 17  5 20 18  4 22 11 23]
---- beds ----
4.273299022456039 4.0
[ 0.  5.  2.  1.  3.  7.  4.  6. nan]
---- livings ----
1.7240710298256308 1.0
[ 0.  3.  1.  2.  4.  5. nan  6.  7.]
---- wc ----
3.723467490446979 4.0
[ 0.  5.  3.  2.  1.  4. nan]
---- area ----
5686.021250616173 350.0
[  816.  1077. 23022. ... 25852. 10595. 10520.]
---- street_width ----
21.786454747861896 20.0
[ 60.  80.  15.  35.  30.  40.  25.  65.  12.   8.  50.  20.   0.  32.
 100.  52.   5.  70.  10.  45.  16.  29.  55.   4.  nan  58.  36.  11.
  22.  24.  21.   7.  42.  31.  95.  17.  14.  28.  64.  26.  27.  13.
   9.  85.  19.  18.  23.  34.   6.  33.  61.  49.  44.  56.  90.  41.
  38.  51.  75.  37.   2.  62.   1.  99.  84.  74.  71.  46.  43.  39.
  59.  54.  72.  47.  57.  79.  53.   3.  63.  69.  81.  83.  48.  82

In [9]:
non_imputable = ['beds', 'livings', 'wc', 'ac', 'ketchen', 'furnished', 'advertiser_type']  # advertiser_type influence a lot

df_notna = df_dropped.copy()
for column_name in non_imputable:
    df_notna = df_notna[df_notna[column_name].notna()]
df_notna

,price,category,beds,livings,wc,area,street_width,age,street_direction,ketchen,...,location.lat,location.lng,user.review,user.iam_verified,rent_period,city_id,district_id,advertiser_type,has_extended_details,daily_rentable
2308,5500.0,1,1.0,1.0,1.0,NaN,NaN,2.0,NaN,1.0,...,24.673321,46.696861,4.56,1.0,2.0,21,506,exclusive_marketer,NaN,0.0
2366,2000.0,1,3.0,1.0,1.0,NaN,NaN,0.0,NaN,1.0,...,20.040100,41.485947,4.70,1.0,2.0,5,4434,agent,NaN,0.0
2369,18000.0,1,5.0,1.0,4.0,3000000.0,NaN,15.0,NaN,1.0,...,27.527849,41.652676,5.00,1.0,3.0,67,2058,exclusive_marketer,NaN,0.0
2405,50000.0,5,7.0,2.0,5.0,900.0,30.0,30.0,8.0,0.0,...,26.361073,43.995518,4.74,1.0,3.0,52,2370,exclusive_marketer,NaN,0.0
2434,1500.0,1,1.0,1.0,1.0,50.0,5.0,NaN,NaN,1.0,...,24.165192,47.322449,4.40,1.0,2.0,13,282,owner,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663918,32000.0,1,4.0,2.0,2.0,160.0,23.0,10.0,NaN,1.0,...,21.649360,39.139800,4.46,1.0,3.0,66,3578,exclusive_marketer,NaN,NaN
663924,35000.0,1,3.0,1.0,2.0,160.0,18.0,9.0,NaN,1.0,...,21.516285,39.226463,5.00,1.0,3.0,66,3604,owner,NaN,NaN
663925,25000.0,1,3.0,1.0,2.0,125.0,NaN,NaN,2.0,0.0,...,24.927849,46.800327,4.03,1.0,3.0,21,446,agent,1.0,NaN
663938,50000.0,1,2.0,2.0,2.0,250.0,5.0,15.0,NaN,1.0,...,24.795143,46.703434,NaN,1.0,3.0,21,522,owner,NaN,NaN


In [10]:
# number of rows lost
df_dropped.shape, df_notna.shape

((663946, 22), (91501, 22))

In [11]:
df_notna.isna().sum()

price                       0
category                    0
beds                        0
livings                     0
wc                          0
area                      329
street_width             1273
age                     11684
street_direction        75541
ketchen                     0
ac                          0
furnished                   0
location.lat                0
location.lng                0
user.review             16586
user.iam_verified         406
rent_period             15022
city_id                     0
district_id                 0
advertiser_type             0
has_extended_details    89375
daily_rentable          61517
dtype: int64

In [19]:
from sklearn.impute import SimpleImputer

# impute special values
df_imputed = df_notna.copy()
df_imputed['has_extended_details'] = df_imputed['has_extended_details'].fillna(0)

# impute by median
median_columns = ['area', 'street_width', 'age', 'street_direction', 'user.review']
for column_name in median_columns:
    median = df_imputed[column_name].median()
    df_imputed[column_name] = df_imputed[column_name].fillna(median)

df_imputed

,price,category,beds,livings,wc,area,street_width,age,street_direction,ketchen,...,location.lat,location.lng,user.review,user.iam_verified,rent_period,city_id,district_id,advertiser_type,has_extended_details,daily_rentable
2308,5500.0,1,1.0,1.0,1.0,160.0,18.0,2.0,3.0,1.0,...,24.673321,46.696861,4.56,1.0,2.0,21,506,exclusive_marketer,0.0,0.0
2366,2000.0,1,3.0,1.0,1.0,160.0,18.0,0.0,3.0,1.0,...,20.040100,41.485947,4.70,1.0,2.0,5,4434,agent,0.0,0.0
2369,18000.0,1,5.0,1.0,4.0,3000000.0,18.0,15.0,3.0,1.0,...,27.527849,41.652676,5.00,1.0,3.0,67,2058,exclusive_marketer,0.0,0.0
2405,50000.0,5,7.0,2.0,5.0,900.0,30.0,30.0,8.0,0.0,...,26.361073,43.995518,4.74,1.0,3.0,52,2370,exclusive_marketer,0.0,0.0
2434,1500.0,1,1.0,1.0,1.0,50.0,5.0,2.0,3.0,1.0,...,24.165192,47.322449,4.40,1.0,2.0,13,282,owner,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663918,32000.0,1,4.0,2.0,2.0,160.0,23.0,10.0,3.0,1.0,...,21.649360,39.139800,4.46,1.0,3.0,66,3578,exclusive_marketer,0.0,NaN
663924,35000.0,1,3.0,1.0,2.0,160.0,18.0,9.0,3.0,1.0,...,21.516285,39.226463,5.00,1.0,3.0,66,3604,owner,0.0,NaN
663925,25000.0,1,3.0,1.0,2.0,125.0,18.0,2.0,2.0,0.0,...,24.927849,46.800327,4.03,1.0,3.0,21,446,agent,1.0,NaN
663938,50000.0,1,2.0,2.0,2.0,250.0,5.0,15.0,3.0,1.0,...,24.795143,46.703434,4.49,1.0,3.0,21,522,owner,0.0,NaN


In [20]:
# find outliers by different values
from sklearn.ensemble import IsolationForest
import numpy as np

possible_outliers = ['area', 'price']

vals = np.array([0] * df_imputed.shape[0])
for column_name in possible_outliers:
    clf = IsolationForest(n_estimators=100, warm_start=False, bootstrap=True, contamination=0.0025, random_state=42)
    vals += clf.fit_predict(df_imputed[column_name].to_numpy().reshape(-1, 1))

In [21]:
df_filtered = df_imputed[vals == 2]

df_imputed[vals != 2]

,price,category,beds,livings,wc,area,street_width,age,street_direction,ketchen,...,location.lat,location.lng,user.review,user.iam_verified,rent_period,city_id,district_id,advertiser_type,has_extended_details,daily_rentable
2369,18000.0,1,5.0,1.0,4.0,3000000.0,18.0,15.0,3.0,1.0,...,27.527849,41.652676,5.00,1.0,3.0,67,2058,exclusive_marketer,0.0,0.0
6080,3200000.0,3,4.0,2.0,4.0,375.0,20.0,4.0,1.0,1.0,...,24.776247,46.655800,4.77,1.0,3.0,21,514,agent,0.0,0.0
7796,3000000.0,7,1.0,0.0,0.0,720.0,40.0,6.0,2.0,1.0,...,24.818441,46.877065,4.38,1.0,NaN,21,410,normal_marketer,0.0,0.0
9781,15000000.0,3,7.0,4.0,5.0,1350.0,20.0,15.0,5.0,1.0,...,24.744841,46.617371,4.33,1.0,3.0,21,596,exclusive_marketer,0.0,0.0
11080,2677500.0,2,2.0,1.0,3.0,450.0,15.0,7.0,1.0,1.0,...,24.810245,46.593656,3.00,1.0,3.0,21,570,normal_marketer,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660843,19200.0,1,4.0,1.0,3.0,14140.0,30.0,2.0,3.0,1.0,...,18.259188,42.764233,5.00,1.0,3.0,70,4060,owner,0.0,NaN
662110,22000.0,1,5.0,1.0,3.0,100000.0,5.0,2.0,3.0,0.0,...,24.516800,39.567139,5.00,1.0,3.0,41,2968,owner,0.0,NaN
662672,2500.0,1,1.0,1.0,2.0,30000.0,5.0,2.0,3.0,1.0,...,18.212112,42.511906,4.49,1.0,2.0,1,3822,owner,0.0,NaN
663485,13200000.0,7,6.0,2.0,5.0,990.0,18.0,2.0,1.0,1.0,...,21.497988,39.272816,4.57,1.0,NaN,66,3426,agent,1.0,NaN


In [22]:
df_filtered

,price,category,beds,livings,wc,area,street_width,age,street_direction,ketchen,...,location.lat,location.lng,user.review,user.iam_verified,rent_period,city_id,district_id,advertiser_type,has_extended_details,daily_rentable
2308,5500.0,1,1.0,1.0,1.0,160.0,18.0,2.0,3.0,1.0,...,24.673321,46.696861,4.56,1.0,2.0,21,506,exclusive_marketer,0.0,0.0
2366,2000.0,1,3.0,1.0,1.0,160.0,18.0,0.0,3.0,1.0,...,20.040100,41.485947,4.70,1.0,2.0,5,4434,agent,0.0,0.0
2405,50000.0,5,7.0,2.0,5.0,900.0,30.0,30.0,8.0,0.0,...,26.361073,43.995518,4.74,1.0,3.0,52,2370,exclusive_marketer,0.0,0.0
2434,1500.0,1,1.0,1.0,1.0,50.0,5.0,2.0,3.0,1.0,...,24.165192,47.322449,4.40,1.0,2.0,13,282,owner,0.0,0.0
2435,1500.0,1,1.0,1.0,1.0,160.0,18.0,2.0,3.0,1.0,...,24.166740,47.325615,4.40,1.0,2.0,13,282,owner,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663918,32000.0,1,4.0,2.0,2.0,160.0,23.0,10.0,3.0,1.0,...,21.649360,39.139800,4.46,1.0,3.0,66,3578,exclusive_marketer,0.0,NaN
663924,35000.0,1,3.0,1.0,2.0,160.0,18.0,9.0,3.0,1.0,...,21.516285,39.226463,5.00,1.0,3.0,66,3604,owner,0.0,NaN
663925,25000.0,1,3.0,1.0,2.0,125.0,18.0,2.0,2.0,0.0,...,24.927849,46.800327,4.03,1.0,3.0,21,446,agent,1.0,NaN
663938,50000.0,1,2.0,2.0,2.0,250.0,5.0,15.0,3.0,1.0,...,24.795143,46.703434,4.49,1.0,3.0,21,522,owner,0.0,NaN


### Data Integration

In [31]:
df_modified = df_filtered.copy()


"""
1= Apartment, rental | 
6= Apartment, sell | 
21= Furnished apartment |

2= Land, sell | 
15= Land, rental |

3= Villa, sell |
5= Villa, rental | 

4= Floor, rental | 
22= Floor, sell | 

7= Building, sell | 
16= Building, rental |

8= Store, rental |
20= Store, sell |

9= House, sell | 
11= House, rental | 

10= Esterahah, sell | 
13= Esterahah, rental | 

12= Farm, sell | 

14= Office, rental | 

17= Warehouse, rental | 

18= Campsite, rental | 

19= Room, rental |

23= Chalet, rental

"""

df_modified['number_of_rooms'] = df_modified['beds'] + df_modified['livings'] + df_modified['wc']

df_modified['is_appartement'] = df_modified['category'].isin([1, 6, 21])
df_modified['is_land'] = df_modified['category'].isin([2, 15])
df_modified['is_villa'] = df_modified['category'].isin([3, 5])
df_modified['is_floot'] = df_modified['category'].isin([4, 22])
df_modified['is_building'] = df_modified['category'].isin([7, 16])
df_modified['is_store'] = df_modified['category'].isin([8, 20])
df_modified['is_house'] = df_modified['category'].isin([9, 11])
df_modified['is_esterahah'] = df_modified['category'].isin([10, 13])
df_modified['is_farm'] = df_modified['category'].isin([12])
df_modified['is_office'] = df_modified['category'].isin([14])
df_modified['is_warehouse'] = df_modified['category'].isin([17])
df_modified['is_campsite'] = df_modified['category'].isin([18])
df_modified['is_room'] = df_modified['category'].isin([19])
df_modified['is_chalet'] = df_modified['category'].isin([23])

df_modified['is_sell'] = df_modified['category'].isin([6, 2, 3, 22, 7, 20, 9, 10, 12])
df_modified['is_rent'] = df_modified['category'].isin([1, 21, 15, 5, 4, 16, 8, 13, 14, 17, 18, 19, 23, 11])

df_modified['all_inclusive'] = \
        (df_modified['beds'] > 0) & \
        (df_modified['livings'] > 0) & \
        (df_modified['wc'] > 0) & \
        (df_modified['ketchen'] > 0) & \
        (df_modified['ac'] > 0) & \
        (df_modified['furnished'] > 0)
        

df_modified

,price,category,beds,livings,wc,area,street_width,age,street_direction,ketchen,...,is_esterahah,is_farm,is_office,is_warehouse,is_campsite,is_room,is_chalet,is_sell,is_rent,all_inclusive
2308,5500.0,1,1.0,1.0,1.0,160.0,18.0,2.0,3.0,1.0,...,False,False,False,False,False,False,False,False,True,True
2366,2000.0,1,3.0,1.0,1.0,160.0,18.0,0.0,3.0,1.0,...,False,False,False,False,False,False,False,False,True,True
2405,50000.0,5,7.0,2.0,5.0,900.0,30.0,30.0,8.0,0.0,...,False,False,False,False,False,False,False,False,True,False
2434,1500.0,1,1.0,1.0,1.0,50.0,5.0,2.0,3.0,1.0,...,False,False,False,False,False,False,False,False,True,True
2435,1500.0,1,1.0,1.0,1.0,160.0,18.0,2.0,3.0,1.0,...,False,False,False,False,False,False,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663918,32000.0,1,4.0,2.0,2.0,160.0,23.0,10.0,3.0,1.0,...,False,False,False,False,False,False,False,False,True,False
663924,35000.0,1,3.0,1.0,2.0,160.0,18.0,9.0,3.0,1.0,...,False,False,False,False,False,False,False,False,True,False
663925,25000.0,1,3.0,1.0,2.0,125.0,18.0,2.0,2.0,0.0,...,False,False,False,False,False,False,False,False,True,False
663938,50000.0,1,2.0,2.0,2.0,250.0,5.0,15.0,3.0,1.0,...,False,False,False,False,False,False,False,False,True,True


### Data Formatting

In [33]:

# categorical features
categorical_columns = ['rent_period', 'advertiser_type', 'daily_rentable', 'user.iam_verified']
df_processed = pd.get_dummies(df_modified, columns=categorical_columns)


In [37]:
df_processed[['price', 'category', 'beds', 'livings', 'wc', 'area', 'street_width',
       'age', 'street_direction', 'ketchen', 'ac', 'furnished', 'location.lat',
       'location.lng', 'user.review', 'city_id', 'district_id',
       'has_extended_details', 'number_of_rooms']]

,price,category,beds,livings,wc,area,street_width,age,street_direction,ketchen,ac,furnished,location.lat,location.lng,user.review,city_id,district_id,has_extended_details,number_of_rooms
2308,5500.0,1,1.0,1.0,1.0,160.0,18.0,2.0,3.0,1.0,1.0,1.0,24.673321,46.696861,4.56,21,506,0.0,3.0
2366,2000.0,1,3.0,1.0,1.0,160.0,18.0,0.0,3.0,1.0,1.0,1.0,20.040100,41.485947,4.70,5,4434,0.0,5.0
2405,50000.0,5,7.0,2.0,5.0,900.0,30.0,30.0,8.0,0.0,0.0,0.0,26.361073,43.995518,4.74,52,2370,0.0,14.0
2434,1500.0,1,1.0,1.0,1.0,50.0,5.0,2.0,3.0,1.0,1.0,1.0,24.165192,47.322449,4.40,13,282,0.0,3.0
2435,1500.0,1,1.0,1.0,1.0,160.0,18.0,2.0,3.0,1.0,1.0,1.0,24.166740,47.325615,4.40,13,282,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663918,32000.0,1,4.0,2.0,2.0,160.0,23.0,10.0,3.0,1.0,0.0,1.0,21.649360,39.139800,4.46,66,3578,0.0,8.0
663924,35000.0,1,3.0,1.0,2.0,160.0,18.0,9.0,3.0,1.0,0.0,0.0,21.516285,39.226463,5.00,66,3604,0.0,6.0
663925,25000.0,1,3.0,1.0,2.0,125.0,18.0,2.0,2.0,0.0,0.0,0.0,24.927849,46.800327,4.03,21,446,1.0,6.0
663938,50000.0,1,2.0,2.0,2.0,250.0,5.0,15.0,3.0,1.0,1.0,1.0,24.795143,46.703434,4.49,21,522,0.0,6.0
